In [1]:
%load_ext autoreload
%autoreload 2
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain import HuggingFacePipeline,HuggingFaceHub
import transformers as ts

In [ ]:
from CustomLLM import LocalLLM
mp='/home/xyy/hfdownloader/aliendao-main/dataroot/models/Qwen/Qwen-7B'
local_llm=LocalLLM.from_pretrain(repo_path=mp)
ts.AutoModelForCausalLM.from_pretrained()

In [ ]:
aaa=ts.AutoTokenizer.from_pretrained('../../model/')